In [223]:
import pandas as pd
from utils import *

config = load_config()

PROJECT_PATH = config.project_path
DATA_PATH = PROJECT_PATH.joinpath("data/raw")

In [224]:
packet12 = pd.read_excel(DATA_PATH.joinpath("packet12.xlsx"), names = ["File Name", "Packet Name", "Concept", "Ranking"])

In [225]:
packet12.loc[:,"File Name"] = packet12["File Name"] + ".txt"
packet12['Ranking'] = packet12.Ranking.apply(lambda x : str(x) + '.0')

In [226]:
msk = packet12.Ranking.str.extract("^(\d+(\.\d+)?)")[0].isna()
packet12 = packet12[~msk]

In [227]:
packet12['Ranking'] = packet12.Ranking.str.extract("^(\d+(\.\d+)?)")[0]

In [228]:
packet12.loc[:,"Ranking"] = packet12.Ranking.astype("float")

In [229]:
def read_packet(packet) :
    df = pd.read_csv(DATA_PATH.joinpath(f"packet_{packet}.txt"), sep="\t")
    df = df[['Packet Name', 'File Name', 'Concept', 'Color', 'Ranking']]
    return df

In [230]:
packetList = ["3A", "3B", "3C", "4A", "4B", "4C", "5B", "5C"]
df_list = []
for packet in packetList :
    df = read_packet("3A")
    df_list.append(df)

In [231]:
packet345 = pd.concat(df_list)

In [232]:
packet12.loc[:,"File Name"] = packet12['File Name'].replace("#", "")

In [233]:
all_packet = pd.concat([packet12,packet345])

In [234]:
all_packet = all_packet.drop(columns = ['Color'])

In [235]:
all_packet['Packet Name'] = all_packet['Packet Name'].str.lower()

In [236]:
consensus_df = []
for file_name in all_packet['File Name'].unique() :
    msk = all_packet[all_packet["File Name"] == file_name].duplicated(subset="Concept", keep=False)
    temp = all_packet[all_packet["File Name"] == file_name][msk]
    temp = temp.groupby(["File Name","Concept"], as_index=False)["Ranking"].mean()
    consensus_df.append(temp)

In [237]:
consensus_df = pd.concat(consensus_df)

In [245]:
# filter 
notes = pd.read_pickle("../data/processed/notes.pkl")
notes.head()

noteid = notes.noteid.unique()

In [251]:
consensus_df = consensus_df[consensus_df["File Name"].isin(noteid)].reset_index(drop=True)

In [259]:
consensus_df.to_pickle(DATA_PATH.joinpath("consensu_df.pkl"))